# Clearwater Modules Architecture

**Author:** Xavier Nogueira
**Overview:**
* How the code organizes a model (static, dynamic, and state variables).
* How the base.Model class handle all shared functionality.

# Installation and Setup

## Install

Carefully follow our **[Installation Instructions](README.md#getting-started)**, especially including:
- Creating a virtual environment for this repository (step 3)

## Import Python Dependancies

In [3]:
import clearwater_modules_python as cwm
from clearwater_modules_python.tsm.model import EnergyBudget
import clearwater_modules_python.sorter as sorter

In [4]:
# Confirm that sub-modules are imported
dir(cwm)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'base',
 'shared',
 'sorter',
 'tsm',
 'utils']

In [5]:
# Confirm that sub-modules are imported
dir(cwm.tsm)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'constants',
 'dynamic_variables',
 'model',
 'state_variables',
 'static_variables',
 'tsm_processes']

### If you get `ModuleNotFoundError`:

If you get this error:
```python
ModuleNotFoundError: No module named 'clearwater_modules_python'
```
Then:
1. Run the following terminal command with your local absolute path to this repo.
    - NOTE: Here we use Jupyter `!` magic command to run from the terminal via this notebook. 
2. Restart the kernel.
3. Rerun the import statements above.

See [4. Add your `ClearWater-modules-python` Path to Miniconda/Anaconda sites-packages](..ReadMe.md#4-add-your-clearwater-modules-python-path-to-minicondaanaconda-sites-packages).

In [6]:
!conda develop /Users/aaufdenkampe/Documents/Python/ClearWater-modules-python/src

path exists, skipping c:\Users\aaufdenkampe\Documents\Python\ClearWater-modules-python\src
completed operation for: c:\Users\aaufdenkampe\Documents\Python\ClearWater-modules-python\src


## Start by instantiating a model
Initial state variable values are always required.

In [7]:
initial_state_values = {
    'water_temp_c': 1.0,
    'volume': 1.0,
    'surface_area': 1.0,
}

In [8]:
my_model = EnergyBudget(
    initial_state_values,
    time_dim='my_time_step',
)
my_model

Initializing from dicts...
Model initialized from input dicts successfully!.


In [9]:
[i for i in dir(my_model) if i[0] != '_']

['all_variables',
 'computation_order',
 'dataset',
 'dynamic_variables',
 'dynamic_variables_names',
 'get_variable',
 'get_variable_names',
 'hotstart_dataset',
 'increment_timestep',
 'initial_state_values',
 'met_parameters',
 'register_variable',
 'state_variables',
 'state_variables_names',
 'static_variable_values',
 'static_variables',
 'static_variables_names',
 'temp_parameters',
 'time_dim',
 'track_dynamic_variables',
 'unregister_variables']

## TSM can be initialized with alternative met/temp parameter
This is an example of a model specific __init__. As of now we are using the defaults.

In [10]:
my_model.met_parameters

{'air_temp_c': 20,
 'q_solar': 400,
 'sed_temp_c': 5.0,
 'eair_mb': 1.0,
 'pressure_mb': 1013.0,
 'cloudiness': 0.1,
 'wind_speed': 3.0,
 'wind_a': 0.3,
 'wind_b': 1.5,
 'wind_c': 1.0,
 'wind_kh_kw': 1.0}

In [11]:
my_model.temp_parameters

{'stefan_boltzmann': 5.67e-08,
 'cp_air': 1005,
 'emissivity_water': 0.97,
 'gravity': -9.806,
 'a0': 6984.505294,
 'a1': -188.903931,
 'a2': 2.133357675,
 'a3': -0.01288580973,
 'a4': 4.393587233e-05,
 'a5': -8.023923082e-08,
 'a6': 6.136820929e-11,
 'pb': 1600.0,
 'cps': 1673.0,
 'h2': 0.1,
 'alphas': 0.0432,
 'richardson_option': True}

In [12]:
my_model.time_dim

'my_time_step'

## All models have static, dynamic, and state variables

In [13]:
display(my_model.static_variables)

[Variable(name='stefan_boltzmann', long_name='Stefan-Boltzmann Constant', units='W m-2 K-4', description='The Stefan-Boltzmann constant.', use='static', process=None),
 Variable(name='cp_air', long_name='Specific Heat Capacity of Air', units='J kg-1 K-1', description='The specific heat capacity of air.', use='static', process=None),
 Variable(name='emissivity_water', long_name='Emissivity of Water', units='1', description='The emissivity of water.', use='static', process=None),
 Variable(name='gravity', long_name='Gravity', units='m s-2', description='The acceleration due to gravity.', use='static', process=None),
 Variable(name='a0', long_name='Albedo of Water', units='unitless', description='The albedo of water.', use='static', process=None),
 Variable(name='a1', long_name='Albedo of Water', units='unitless', description='The albedo of water.', use='static', process=None),
 Variable(name='a2', long_name='Albedo of Water', units='unitless', description='The albedo of water.', use='sta

In [14]:
display(my_model.dynamic_variables)

[Variable(name='air_temp_k', long_name='Air temperature', units='K', description='Air temperature', use='dynamic', process=CPUDispatcher(<function air_temp_k at 0x000001B2A7BF20C0>)),
 Variable(name='water_temp_k', long_name='Water temperature', units='K', description='Water temperature', use='dynamic', process=CPUDispatcher(<function water_temp_k at 0x000001B2A7BCBD80>)),
 Variable(name='mixing_ratio_air', long_name='Mixing ratio of air', units='unitless', description='Mixing ratio of air', use='dynamic', process=CPUDispatcher(<function mixing_ratio_air at 0x000001B2A7BF2160>)),
 Variable(name='density_air', long_name='Density of air', units='kg/m^3', description='Density of air', use='dynamic', process=CPUDispatcher(<function density_air at 0x000001B2A7BF2340>)),
 Variable(name='density_water', long_name='Density of water', units='kg/m^3', description='Density of water', use='dynamic', process=CPUDispatcher(<function mf_density_water at 0x000001B2A7C08E00>)),
 Variable(name='esat_mb'

In [15]:
display(my_model.state_variables)

[Variable(name='water_temp_c', long_name='Water temperature', units='degC', description='TSM state variable for water temperature', use='state', process=CPUDispatcher(<function t_water_c at 0x000001B2A7C09BC0>)),
 Variable(name='surface_area', long_name='Surface area', units='m^2', description='Surface area', use='state', process=<function mock_equation at 0x000001B2A7BF0CC0>),
 Variable(name='volume', long_name='Volume', units='m^3', description='Volume', use='state', process=<function mock_equation at 0x000001B2A7BF0CC0>)]

## One can access their "computation order" which is calculated using a "dependency tree" approach in `sorter.py`

In [16]:
my_model.computation_order

[Variable(name='air_temp_k', long_name='Air temperature', units='K', description='Air temperature', use='dynamic', process=CPUDispatcher(<function air_temp_k at 0x000001B2A7BF20C0>)),
 Variable(name='water_temp_k', long_name='Water temperature', units='K', description='Water temperature', use='dynamic', process=CPUDispatcher(<function water_temp_k at 0x000001B2A7BCBD80>)),
 Variable(name='mixing_ratio_air', long_name='Mixing ratio of air', units='unitless', description='Mixing ratio of air', use='dynamic', process=CPUDispatcher(<function mixing_ratio_air at 0x000001B2A7BF2160>)),
 Variable(name='density_air', long_name='Density of air', units='kg/m^3', description='Density of air', use='dynamic', process=CPUDispatcher(<function density_air at 0x000001B2A7BF2340>)),
 Variable(name='density_water', long_name='Density of water', units='kg/m^3', description='Density of water', use='dynamic', process=CPUDispatcher(<function mf_density_water at 0x000001B2A7C08E00>)),
 Variable(name='esat_mb'

In [17]:
for i in my_model.computation_order:
    print(f'{i.name} | {sorter.get_process_args(i.process)}')

air_temp_k | ['air_temp_c']
water_temp_k | ['water_temp_c']
mixing_ratio_air | ['eair_mb', 'pressure_mb']
density_air | ['pressure_mb', 'air_temp_k', 'mixing_ratio_air']
density_water | ['water_temp_c']
esat_mb | ['water_temp_k', 'a0', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6']
density_air_sat | ['water_temp_k', 'esat_mb', 'pressure_mb']
ri_number | ['gravity', 'density_air', 'density_air_sat', 'wind_speed']
ri_function | ['ri_number']
lv | ['water_temp_k']
cp_water | ['water_temp_c']
emissivity_air | ['air_temp_k']
wind_function | ['wind_a', 'wind_b', 'wind_c', 'wind_speed']
q_latent | ['ri_function', 'pressure_mb', 'density_water', 'lv', 'wind_function', 'esat_mb', 'eair_mb']
q_sensible | ['wind_kh_kw', 'ri_function', 'cp_air', 'density_water', 'wind_function', 'air_temp_k', 'water_temp_k']
q_sediment | ['pb', 'cps', 'alphas', 'h2', 'sed_temp_c', 'water_temp_c']
dTdt_sediment_c | ['alphas', 'h2', 'water_temp_c', 'sed_temp_c']
q_longwave_down | ['air_temp_k', 'emissivity_air', 'cloudiness', 

## Data is stored in `self.dataset`

In [18]:
my_model.dataset

<xarray.Dataset>
Dimensions:            (my_time_step: 1, x: 1, y: 1)
Coordinates:
  * my_time_step       (my_time_step) int32 0
  * x                  (x) float64 1.0
  * y                  (y) float64 1.0
Data variables: (12/30)
    water_temp_c       (my_time_step, x, y) float64 1.0
    volume             (my_time_step, x, y) float64 1.0
    surface_area       (my_time_step, x, y) float64 1.0
    stefan_boltzmann   (x, y) float64 5.67e-08
    cp_air             (x, y) int32 1005
    emissivity_water   (x, y) float64 0.97
    ...                 ...
    cloudiness         (x, y) float64 0.1
    wind_speed         (x, y) float64 3.0
    wind_a             (x, y) float64 0.3
    wind_b             (x, y) float64 1.5
    wind_c             (x, y) float64 1.0
    wind_kh_kw         (x, y) float64 1.0

## Running a timestep
All timesteps can be run independently. Optionally, one can update the state values with a float or a `xarray.DataArray`.

In [19]:
%%time
my_model.increment_timestep()

CPU times: total: 5.41 s
Wall time: 7.21 s


<xarray.Dataset>
Dimensions:            (my_time_step: 2, x: 1, y: 1)
Coordinates:
  * my_time_step       (my_time_step) int32 0 1
  * x                  (x) float64 1.0
  * y                  (y) float64 1.0
Data variables: (12/51)
    water_temp_c       (my_time_step, x, y) float64 1.0 0.9299
    volume             (my_time_step, x, y) float64 1.0 1.0
    surface_area       (my_time_step, x, y) float64 1.0 1.0
    stefan_boltzmann   (x, y) float64 5.67e-08
    cp_air             (x, y) int32 1005
    emissivity_water   (x, y) float64 0.97
    ...                 ...
    q_sediment         (my_time_step, x, y) float64 nan 107.1
    dTdt_sediment_c    (my_time_step, x, y) float64 nan -34.56
    q_longwave_down    (my_time_step, x, y) float64 nan 337.8
    q_longwave_up      (my_time_step, x, y) float64 nan 310.7
    q_net              (my_time_step, x, y) float64 nan -140.2
    dTdt_water_c       (my_time_step, x, y) float64 nan -0.07009

In [20]:
my_model.dataset

<xarray.Dataset>
Dimensions:            (my_time_step: 2, x: 1, y: 1)
Coordinates:
  * my_time_step       (my_time_step) int32 0 1
  * x                  (x) float64 1.0
  * y                  (y) float64 1.0
Data variables: (12/51)
    water_temp_c       (my_time_step, x, y) float64 1.0 0.9299
    volume             (my_time_step, x, y) float64 1.0 1.0
    surface_area       (my_time_step, x, y) float64 1.0 1.0
    stefan_boltzmann   (x, y) float64 5.67e-08
    cp_air             (x, y) int32 1005
    emissivity_water   (x, y) float64 0.97
    ...                 ...
    q_sediment         (my_time_step, x, y) float64 nan 107.1
    dTdt_sediment_c    (my_time_step, x, y) float64 nan -34.56
    q_longwave_down    (my_time_step, x, y) float64 nan 337.8
    q_longwave_up      (my_time_step, x, y) float64 nan 310.7
    q_net              (my_time_step, x, y) float64 nan -140.2
    dTdt_water_c       (my_time_step, x, y) float64 nan -0.07009

## Speed running (i need to play with numba a bit to boost speed)

In [21]:
import numba
TIME_STEPS = 10

In [22]:
@numba.jit(forceobj=True)
def run_n_timesteps(time_steps: int, model: EnergyBudget):
    for i in range(time_steps):
        model.increment_timestep()

In [23]:
run_n_timesteps(TIME_STEPS, my_model)